In [1]:
import os
import sys

import random
import math
import numpy as np
import cv2
import matplotlib.pyplot as plt
import json
import pydicom
from imgaug import augmenters as iaa
from tqdm import tqdm
import pandas as pd
import glob
import scipy.signal as signal
#import skimage.morphology as sm

# enter your Kaggle credentionals here
os.environ['KAGGLE_USERNAME']="lixiaoqi"
os.environ['KAGGLE_KEY']="a96b56107f574e1cf4e14e3822ca2c93"

os.chdir('E:\学习\ml-lessons')
# Root directory of the project
ROOT_DIR = os.path.abspath('./lesson3-data')

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, 'logs')

#if not os.path.exists(ROOT_DIR):
#    os.makedirs(ROOT_DIR)
os.chdir(ROOT_DIR)####

# Import Mask RCNN
sys.path.append(os.path.join(ROOT_DIR, 'Mask_RCNN'))  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

train_dicom_dir = os.path.join(ROOT_DIR, 'stage_1_train_images')
test_dicom_dir = os.path.join(ROOT_DIR, 'stage_1_test_images')

'''
- dicom_fps is a list of the dicom image path and filenames 
- image_annotions is a dictionary of the annotations keyed by the filenames
- parsing the dataset returns a list of the image filenames and the annotations dictionary
'''
# 将图片的绝对路径以list方式保存起来，这样就可以通过索引的方式直接获取图像路径及名称
def get_dicom_fps(dicom_dir):
    dicom_fps = glob.glob(dicom_dir+'/'+'*.dcm')
    return list(set(dicom_fps))

#image_fps保存了list类型的图像路径及名称，image_annotations保存了每一张图像所对应的识别框的信息
def parse_dataset(dicom_dir, anns): 
    image_fps = get_dicom_fps(dicom_dir)
    image_annotations = {fp: [] for fp in image_fps}
    for index, row in anns.iterrows(): 
        fp = os.path.join(dicom_dir, row['patientId']+'.dcm')
        image_annotations[fp].append(row)
    return image_fps, image_annotations 

# set color for class
def get_colors_for_class_ids(class_ids):
    colors = []
    for class_id in class_ids:
        if class_id == 1:
            colors.append((.941, .204, .204))
    return colors

#  直方图均衡
def histep(img, nbr_bins=256):
    """Histogram equalization of a grayscale image."""
    # 获取直方图p(r)
    imhist, bins = np.histogram(img.flatten(), nbr_bins, normed=True)
    
    # 获取T(r)
    cdf = imhist.cumsum()  # cumulative distribution function
    cdf = 255 * cdf / cdf[-1]
    
    #获取s, 并用s替换原始图像对应的灰度值
    result = np.interp(img.flatten(), bins[:-1], cdf)
    return result.reshape(img.shape), cdf

# The following parameters have been selected to reduce running time for demonstration purposes 
# These are not optimal 
# 为在RSNA pneumonia数据集上训练模型而重写mask_rcnn中的Config类
class DetectorConfig(Config):
    # 给该configuration取一个可识别的名字
    NAME = 'pneumonia'
    
    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 4#4,8
    #BATCH_SIZE是上面两个值的乘积
    BACKBONE = 'resnet50'
    
    NUM_CLASSES = 2   # background + 1 pneumonia classes
    
    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 512  ####64,256,512
    IMAGE_MAX_DIM = 512
    
    RPN_ANCHOR_SCALES = (32, 64, 128, 256)#(16, 32, 64, 128)   #(32, 64, 128, 256)
    
    TRAIN_ROIS_PER_IMAGE = 32  #16,32
    
    MAX_GT_INSTANCES = 4
    
    DETECTION_MAX_INSTANCES = 3
    DETECTION_MIN_CONFIDENCE = 0.7   #待定
    DETECTION_NMS_THRESHOLD = 0.01
    
    # RPN_TRAIN_ANCHORS_PER_IMAGE = 16
    STEPS_PER_EPOCH = 400  #100
    # TOP_DOWN_PYRAMID_SIZE = 32
    
config = DetectorConfig()
#config.display()

class DetectorDataset(utils.Dataset):
    """Dataset class for training pneumonia detection on the RSNA pneumonia dataset.
    """

    def __init__(self, image_fps, image_annotations, orig_height, orig_width):
        super().__init__(self)
        
        # 添加类
        self.add_class('pneumonia', 1, 'Lung Opacity')
   
        # 添加图像
        for i, fp in enumerate(image_fps):
            annotations = image_annotations[fp]
            self.add_image('pneumonia', image_id=i, path=fp, 
                           annotations=annotations, orig_height=orig_height, orig_width=orig_width)
            
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']

    def load_image(self, image_id):
        info = self.image_info[image_id]
        fp = info['path']
        ds = pydicom.read_file(fp)
        image = ds.pixel_array
        
        # image = signal.medfilt(image, kernel_size=5)   # 对图片进行中值滤波
        # image = image.astype(np.uint8)
        # image = cv2.equalizeHist(image)  # 对图片进行直方图均衡处理
        #image = sm.closing(image,sm.disk(5))  #用边长为5的圆形滤波器进行闭运算滤波
        
        image, _ = histep(image)   # 对图片进行直方图均衡处理
        # If grayscale. Convert to RGB for consistency.
        if len(image.shape) != 3 or image.shape[2] != 3:
            image = np.stack((image,) * 3, -1)
        return image

    def load_mask(self, image_id):
        info = self.image_info[image_id]
        annotations = info['annotations']
        count = len(annotations)
        if count == 0:
            mask = np.zeros((info['orig_height'], info['orig_width'], 1), dtype=np.uint8)
            class_ids = np.zeros((1,), dtype=np.int32)
        else:
            mask = np.zeros((info['orig_height'], info['orig_width'], count), dtype=np.uint8)
            class_ids = np.zeros((count,), dtype=np.int32)
            for i, a in enumerate(annotations):
                if a['Target'] == 1:
                    x = int(a['x'])
                    y = int(a['y'])
                    w = int(a['width'])
                    h = int(a['height'])
                    mask_instance = mask[:, :, i].copy()
                    cv2.rectangle(mask_instance, (x, y), (x+w, y+h), 255, -1)
                    mask[:, :, i] = mask_instance
                    class_ids[i] = 1
        return mask.astype(np.bool), class_ids.astype(np.int32)

    
# 训练集
f=open(os.path.join(ROOT_DIR, 'stage_1_train_labels.csv'))
anns = pd.read_csv(f)
image_fps, image_annotations = parse_dataset(train_dicom_dir, anns=anns)
# Original DICOM image size: 1024 x 1024
ORIG_SIZE = 1024

# Modify this line to use more or fewer images for training/validation. 
# To use all images, do: image_fps_list = list(image_fps)
image_fps_list = list(image_fps)#[:1000] 
# 将数据分为训练集和验证集
# split ratio is set to 0.9 vs. 0.1 (train vs. validation, respectively)
#sorted(image_fps_list)
random.seed(42)
random.shuffle(image_fps_list)

# 训练集和验证集
val_size = 1500
image_fps_val = image_fps_list[:val_size]
image_fps_train = image_fps_list[val_size:]
'''
validation_split = 0.1
split_index = int((1 - validation_split) * len(image_fps_list))
image_fps_train = image_fps_list[:split_index]
image_fps_val = image_fps_list[split_index:]
'''
print('训练和测试样例个数:',len(image_fps_train), len(image_fps_val))

# prepare the training dataset
dataset_train = DetectorDataset(image_fps_train, image_annotations, ORIG_SIZE, ORIG_SIZE)
dataset_train.prepare()

# Show annotation(s) for a DICOM image 
# choice() 方法返回一个列表，元组或字符串的随机项。
# test_fp = random.choice(image_fps_train)
# print(image_annotations[test_fp])

# prepare the validation dataset
dataset_val = DetectorDataset(image_fps_val, image_annotations, ORIG_SIZE, ORIG_SIZE)
dataset_val.prepare()

model = modellib.MaskRCNN(mode='training', config=config, model_dir=MODEL_DIR)

# Image augmentation (light but constant)
'''augmentation = iaa.Sequential([
        iaa.Fliplr(0.5),
        iaa.Affine(
            scale={"x": (0.9, 1.1), "y": (0.9, 1.1)},
            translate_percent={"x": (-0.04, 0.04), "y": (-0.08, 0.08)},
            rotate=(-5, 5),
            shear=(-1, 1)
        )
    
])'''
augmentation = iaa.Sequential([
        iaa.Fliplr(0.5),
        iaa.Affine(
                scale={"x": (0.98, 1.02), "y": (0.98, 1.02)},
                translate_percent={"x": (-0.02, 0.02), "y": (-0.04, 0.04)},
                rotate=(-2, 2),
                shear=(-1, 1),
            ),
    
])
'''augmentation = iaa.Sequential([
        iaa.Fliplr(0.5),
        iaa.OneOf([  # geometric transform
            iaa.Affine(
                scale={"x": (0.98, 1.02), "y": (0.98, 1.02)},
                translate_percent={"x": (-0.02, 0.02), "y": (-0.04, 0.04)},
                rotate=(-2, 2),
                shear=(-1, 1),
            ),
            #iaa.PiecewiseAffine(scale=(0.001, 0.025)),
        ]),
        iaa.OneOf([  # brightness or contrast
            iaa.Multiply((0.9, 1.1)),
            iaa.ContrastNormalization((0.9, 1.1)),
        ]),
        iaa.OneOf([  # blur or sharpen
            iaa.GaussianBlur(sigma=(0.0, 0.1)),
            iaa.Sharpen(alpha=(0.0, 0.1)),
        ])
])'''
'''
# 加载预训练的COCO权重
COCO_MODEL_PATH = "./Mask_RCNN/samples/coco/mask_rcnn_coco.h5"

#设定权重的初始化方式，有imagenet，coco,last三种
init_with = "coco"
if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    model.load_weights(model.find_last()[1], by_name=True)
'''

model_path = 'E:\学习\ml-lessons\lesson3-data\logs\pneumonia20181013T2344\mask_rcnn_pneumonia_0040.h5'
model.load_weights(model_path, by_name=True)

NUM_EPOCHS = 33
LEARNING_RATE = 0.005
# Train Mask-RCNN Model 
import warnings 
warnings.filterwarnings("ignore")
'''
# 头两个epoch使用较大的learning rate，并且只训练heads部分
model.train(dataset_train, dataset_val, 
            learning_rate=LEARNING_RATE*2, 
            epochs=2, 
            layers='heads',
            augmentation=None)  # no need to augment yet
model.train(dataset_train, dataset_val, 
            learning_rate=LEARNING_RATE, 
            epochs=6, 
            layers='all',
            augmentation=augmentation)
# 后面的epoch使用更小的learning rate
model.train(dataset_train, dataset_val, 
            learning_rate=LEARNING_RATE/5, 
            epochs=16, 
            layers='all',
            augmentation=augmentation)
model.train(dataset_train, dataset_val, 
            learning_rate=LEARNING_RATE/50, 
            epochs=NUM_EPOCHS,
            layers='all',
            augmentation=augmentation)'''
model.train(dataset_train, dataset_val, 
            learning_rate=LEARNING_RATE/500, 
            epochs=40,
            layers='all',
            augmentation=augmentation)

print('train complete')

Using TensorFlow backend.


训练和测试样例个数: 24184 1500
Re-starting from epoch 30

Starting at epoch 30. LR=1e-05

Checkpoint Path: E:\学习\ml-lessons\lesson3-data\logs\pneumonia20181013T2344\mask_rcnn_pneumonia_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a 

 53/400 [==>...........................] - ETA: 2:45:52 - loss: 1.0850 - rpn_class_loss: 0.0024 - rpn_bbox_loss: 0.2172 - mrcnn_class_loss: 0.1312 - mrcnn_bbox_loss: 0.3874 - mrcnn_mask_loss: 0.34 - ETA: 1:37:21 - loss: 0.9855 - rpn_class_loss: 0.0036 - rpn_bbox_loss: 0.2004 - mrcnn_class_loss: 0.1318 - mrcnn_bbox_loss: 0.3172 - mrcnn_mask_loss: 0.33 - ETA: 1:12:30 - loss: 0.9752 - rpn_class_loss: 0.0055 - rpn_bbox_loss: 0.2066 - mrcnn_class_loss: 0.1372 - mrcnn_bbox_loss: 0.3082 - mrcnn_mask_loss: 0.31 - ETA: 1:04:25 - loss: 0.9967 - rpn_class_loss: 0.0051 - rpn_bbox_loss: 0.1992 - mrcnn_class_loss: 0.1468 - mrcnn_bbox_loss: 0.3116 - mrcnn_mask_loss: 0.33 - ETA: 55:33 - loss: 0.9723 - rpn_class_loss: 0.0051 - rpn_bbox_loss: 0.1947 - mrcnn_class_loss: 0.1457 - mrcnn_bbox_loss: 0.2988 - mrcnn_mask_loss: 0.3280 - ETA: 49:38 - loss: 0.9938 - rpn_class_loss: 0.0060 - rpn_bbox_loss: 0.2172 - mrcnn_class_loss: 0.1460 - mrcnn_bbox_loss: 0.2942 - mrcnn_mask_loss: 0.33 - ETA: 47:20 - loss: 1.08

212/400 [==============>...............] - ETA: 17:42 - loss: 1.1189 - rpn_class_loss: 0.0081 - rpn_bbox_loss: 0.2447 - mrcnn_class_loss: 0.1756 - mrcnn_bbox_loss: 0.3331 - mrcnn_mask_loss: 0.35 - ETA: 17:37 - loss: 1.1194 - rpn_class_loss: 0.0081 - rpn_bbox_loss: 0.2442 - mrcnn_class_loss: 0.1759 - mrcnn_bbox_loss: 0.3333 - mrcnn_mask_loss: 0.35 - ETA: 17:33 - loss: 1.1183 - rpn_class_loss: 0.0081 - rpn_bbox_loss: 0.2438 - mrcnn_class_loss: 0.1754 - mrcnn_bbox_loss: 0.3331 - mrcnn_mask_loss: 0.35 - ETA: 17:28 - loss: 1.1181 - rpn_class_loss: 0.0080 - rpn_bbox_loss: 0.2440 - mrcnn_class_loss: 0.1755 - mrcnn_bbox_loss: 0.3328 - mrcnn_mask_loss: 0.35 - ETA: 17:24 - loss: 1.1185 - rpn_class_loss: 0.0080 - rpn_bbox_loss: 0.2442 - mrcnn_class_loss: 0.1755 - mrcnn_bbox_loss: 0.3333 - mrcnn_mask_loss: 0.35 - ETA: 17:20 - loss: 1.1174 - rpn_class_loss: 0.0080 - rpn_bbox_loss: 0.2439 - mrcnn_class_loss: 0.1753 - mrcnn_bbox_loss: 0.3329 - mrcnn_mask_loss: 0.35 - ETA: 17:14 - loss: 1.1192 - rpn_c

265/400 [==================>...........] - ETA: 13:44 - loss: 1.1242 - rpn_class_loss: 0.0081 - rpn_bbox_loss: 0.2435 - mrcnn_class_loss: 0.1803 - mrcnn_bbox_loss: 0.3343 - mrcnn_mask_loss: 0.35 - ETA: 13:41 - loss: 1.1243 - rpn_class_loss: 0.0081 - rpn_bbox_loss: 0.2432 - mrcnn_class_loss: 0.1805 - mrcnn_bbox_loss: 0.3344 - mrcnn_mask_loss: 0.35 - ETA: 13:38 - loss: 1.1270 - rpn_class_loss: 0.0082 - rpn_bbox_loss: 0.2436 - mrcnn_class_loss: 0.1813 - mrcnn_bbox_loss: 0.3353 - mrcnn_mask_loss: 0.35 - ETA: 13:33 - loss: 1.1275 - rpn_class_loss: 0.0083 - rpn_bbox_loss: 0.2434 - mrcnn_class_loss: 0.1823 - mrcnn_bbox_loss: 0.3350 - mrcnn_mask_loss: 0.35 - ETA: 13:28 - loss: 1.1276 - rpn_class_loss: 0.0083 - rpn_bbox_loss: 0.2429 - mrcnn_class_loss: 0.1828 - mrcnn_bbox_loss: 0.3351 - mrcnn_mask_loss: 0.35 - ETA: 13:25 - loss: 1.1268 - rpn_class_loss: 0.0083 - rpn_bbox_loss: 0.2424 - mrcnn_class_loss: 0.1827 - mrcnn_bbox_loss: 0.3351 - mrcnn_mask_loss: 0.35 - ETA: 13:20 - loss: 1.1272 - rpn_c

...

212/400 [==============>...............] - ETA: 16:08 - loss: 1.1280 - rpn_class_loss: 0.0088 - rpn_bbox_loss: 0.2538 - mrcnn_class_loss: 0.1815 - mrcnn_bbox_loss: 0.3284 - mrcnn_mask_loss: 0.35 - ETA: 16:10 - loss: 1.1278 - rpn_class_loss: 0.0087 - rpn_bbox_loss: 0.2534 - mrcnn_class_loss: 0.1816 - mrcnn_bbox_loss: 0.3285 - mrcnn_mask_loss: 0.35 - ETA: 16:05 - loss: 1.1273 - rpn_class_loss: 0.0087 - rpn_bbox_loss: 0.2531 - mrcnn_class_loss: 0.1816 - mrcnn_bbox_loss: 0.3285 - mrcnn_mask_loss: 0.35 - ETA: 16:03 - loss: 1.1262 - rpn_class_loss: 0.0087 - rpn_bbox_loss: 0.2524 - mrcnn_class_loss: 0.1815 - mrcnn_bbox_loss: 0.3285 - mrcnn_mask_loss: 0.35 - ETA: 16:00 - loss: 1.1265 - rpn_class_loss: 0.0087 - rpn_bbox_loss: 0.2531 - mrcnn_class_loss: 0.1813 - mrcnn_bbox_loss: 0.3285 - mrcnn_mask_loss: 0.35 - ETA: 15:58 - loss: 1.1255 - rpn_class_loss: 0.0087 - rpn_bbox_loss: 0.2524 - mrcnn_class_loss: 0.1814 - mrcnn_bbox_loss: 0.3281 - mrcnn_mask_loss: 0.35 - ETA: 15:56 - loss: 1.1251 - rpn_c

265/400 [==================>...........] - ETA: 12:40 - loss: 1.1214 - rpn_class_loss: 0.0086 - rpn_bbox_loss: 0.2530 - mrcnn_class_loss: 0.1781 - mrcnn_bbox_loss: 0.3271 - mrcnn_mask_loss: 0.35 - ETA: 12:36 - loss: 1.1211 - rpn_class_loss: 0.0086 - rpn_bbox_loss: 0.2527 - mrcnn_class_loss: 0.1783 - mrcnn_bbox_loss: 0.3270 - mrcnn_mask_loss: 0.35 - ETA: 12:31 - loss: 1.1205 - rpn_class_loss: 0.0086 - rpn_bbox_loss: 0.2525 - mrcnn_class_loss: 0.1780 - mrcnn_bbox_loss: 0.3269 - mrcnn_mask_loss: 0.35 - ETA: 12:27 - loss: 1.1210 - rpn_class_loss: 0.0086 - rpn_bbox_loss: 0.2526 - mrcnn_class_loss: 0.1780 - mrcnn_bbox_loss: 0.3271 - mrcnn_mask_loss: 0.35 - ETA: 12:21 - loss: 1.1215 - rpn_class_loss: 0.0086 - rpn_bbox_loss: 0.2524 - mrcnn_class_loss: 0.1786 - mrcnn_bbox_loss: 0.3273 - mrcnn_mask_loss: 0.35 - ETA: 12:17 - loss: 1.1220 - rpn_class_loss: 0.0085 - rpn_bbox_loss: 0.2523 - mrcnn_class_loss: 0.1790 - mrcnn_bbox_loss: 0.3274 - mrcnn_mask_loss: 0.35 - ETA: 12:12 - loss: 1.1223 - rpn_c

318/400 [======================>.......] - ETA: 9:04 - loss: 1.1200 - rpn_class_loss: 0.0085 - rpn_bbox_loss: 0.2520 - mrcnn_class_loss: 0.1793 - mrcnn_bbox_loss: 0.3263 - mrcnn_mask_loss: 0.354 - ETA: 8:59 - loss: 1.1213 - rpn_class_loss: 0.0085 - rpn_bbox_loss: 0.2521 - mrcnn_class_loss: 0.1795 - mrcnn_bbox_loss: 0.3269 - mrcnn_mask_loss: 0.354 - ETA: 8:55 - loss: 1.1199 - rpn_class_loss: 0.0085 - rpn_bbox_loss: 0.2515 - mrcnn_class_loss: 0.1795 - mrcnn_bbox_loss: 0.3264 - mrcnn_mask_loss: 0.354 - ETA: 8:52 - loss: 1.1199 - rpn_class_loss: 0.0084 - rpn_bbox_loss: 0.2513 - mrcnn_class_loss: 0.1796 - mrcnn_bbox_loss: 0.3265 - mrcnn_mask_loss: 0.354 - ETA: 8:49 - loss: 1.1199 - rpn_class_loss: 0.0084 - rpn_bbox_loss: 0.2516 - mrcnn_class_loss: 0.1798 - mrcnn_bbox_loss: 0.3262 - mrcnn_mask_loss: 0.353 - ETA: 8:45 - loss: 1.1228 - rpn_class_loss: 0.0085 - rpn_bbox_loss: 0.2541 - mrcnn_class_loss: 0.1793 - mrcnn_bbox_loss: 0.3268 - mrcnn_mask_loss: 0.354 - ETA: 8:41 - loss: 1.1223 - rpn_cl

371/400 [==========================>...] - ETA: 5:30 - loss: 1.1194 - rpn_class_loss: 0.0086 - rpn_bbox_loss: 0.2524 - mrcnn_class_loss: 0.1788 - mrcnn_bbox_loss: 0.3251 - mrcnn_mask_loss: 0.354 - ETA: 5:26 - loss: 1.1214 - rpn_class_loss: 0.0087 - rpn_bbox_loss: 0.2540 - mrcnn_class_loss: 0.1789 - mrcnn_bbox_loss: 0.3253 - mrcnn_mask_loss: 0.354 - ETA: 5:21 - loss: 1.1228 - rpn_class_loss: 0.0087 - rpn_bbox_loss: 0.2546 - mrcnn_class_loss: 0.1795 - mrcnn_bbox_loss: 0.3255 - mrcnn_mask_loss: 0.354 - ETA: 5:17 - loss: 1.1217 - rpn_class_loss: 0.0087 - rpn_bbox_loss: 0.2540 - mrcnn_class_loss: 0.1796 - mrcnn_bbox_loss: 0.3251 - mrcnn_mask_loss: 0.354 - ETA: 5:13 - loss: 1.1206 - rpn_class_loss: 0.0087 - rpn_bbox_loss: 0.2535 - mrcnn_class_loss: 0.1793 - mrcnn_bbox_loss: 0.3250 - mrcnn_mask_loss: 0.354 - ETA: 5:10 - loss: 1.1212 - rpn_class_loss: 0.0087 - rpn_bbox_loss: 0.2533 - mrcnn_class_loss: 0.1795 - mrcnn_bbox_loss: 0.3254 - mrcnn_mask_loss: 0.354 - ETA: 5:05 - loss: 1.1207 - rpn_cl

400/400 [==============================] - ETA: 1:54 - loss: 1.1378 - rpn_class_loss: 0.0090 - rpn_bbox_loss: 0.2644 - mrcnn_class_loss: 0.1798 - mrcnn_bbox_loss: 0.3282 - mrcnn_mask_loss: 0.356 - ETA: 1:50 - loss: 1.1378 - rpn_class_loss: 0.0090 - rpn_bbox_loss: 0.2645 - mrcnn_class_loss: 0.1796 - mrcnn_bbox_loss: 0.3283 - mrcnn_mask_loss: 0.356 - ETA: 1:46 - loss: 1.1373 - rpn_class_loss: 0.0090 - rpn_bbox_loss: 0.2642 - mrcnn_class_loss: 0.1796 - mrcnn_bbox_loss: 0.3281 - mrcnn_mask_loss: 0.356 - ETA: 1:42 - loss: 1.1374 - rpn_class_loss: 0.0090 - rpn_bbox_loss: 0.2640 - mrcnn_class_loss: 0.1796 - mrcnn_bbox_loss: 0.3283 - mrcnn_mask_loss: 0.356 - ETA: 1:38 - loss: 1.1374 - rpn_class_loss: 0.0090 - rpn_bbox_loss: 0.2639 - mrcnn_class_loss: 0.1797 - mrcnn_bbox_loss: 0.3283 - mrcnn_mask_loss: 0.356 - ETA: 1:34 - loss: 1.1374 - rpn_class_loss: 0.0090 - rpn_bbox_loss: 0.2638 - mrcnn_class_loss: 0.1798 - mrcnn_bbox_loss: 0.3284 - mrcnn_mask_loss: 0.356 - ETA: 1:30 - loss: 1.1371 - rpn_cl

In [2]:
# select trained model 
dir_names = next(os.walk(model.model_dir))[1]
key = config.NAME.lower()
dir_names = filter(lambda f: f.startswith(key), dir_names)
dir_names = sorted(dir_names)

if not dir_names:
    import errno
    raise FileNotFoundError(
            errno.ENOENT,
            "Could not find model directory under {}".format(model.model_dir))          
    
fps = []
# Pick last directory
for d in dir_names: 
    dir_name = os.path.join(model.model_dir, d)
    # Find the last checkpoint
    checkpoints = next(os.walk(dir_name))[2]
    checkpoints = filter(lambda f: f.startswith("mask_rcnn"), checkpoints)
    checkpoints = sorted(checkpoints)
    if not checkpoints:
        print('No weight files in {}'.format(dir_name))
    else: 
        checkpoint = os.path.join(dir_name, checkpoints[-1])
        fps.append(checkpoint)

model_path = sorted(fps)[-1]
print('Found model {}'.format(model_path))

Found model E:\学习\ml-lessons\lesson3-data\logs\pneumonia20181013T2344\mask_rcnn_pneumonia_0040.h5


In [7]:
model_path = 'E:\学习\ml-lessons\lesson3-data\logs\pneumonia20181013T2344\mask_rcnn_pneumonia_0037.h5'

class InferenceConfig(DetectorConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode='inference', 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

# Get filenames of test dataset DICOM images
test_image_fps = get_dicom_fps(test_dicom_dir)

# Make predictions on test images, write out sample submission 
def predict(image_fps, filepath='submission.csv', min_conf=0.95): 
    
    # assume square image
    resize_factor = ORIG_SIZE / config.IMAGE_SHAPE[0]
    with open(filepath, 'w') as file:
      file.write("patientId,PredictionString\n")
      
      for image_id in tqdm(image_fps): 
        ds = pydicom.read_file(image_id)
        image = ds.pixel_array
        # image = signal.medfilt(image, kernel_size=5)   # 对图片进行中值滤波
        # image = image.astype(np.uint8)
        # image = cv2.equalizeHist(image)  # 对图片进行直方图均衡处理
        image, _ = histep(image)   # 对图片进行直方图均衡处理
          
        # If grayscale. Convert to RGB for consistency.
        if len(image.shape) != 3 or image.shape[2] != 3:
            image = np.stack((image,) * 3, -1) 
        image, window, scale, padding, crop = utils.resize_image(
            image,
            min_dim=config.IMAGE_MIN_DIM,
            min_scale=config.IMAGE_MIN_SCALE,
            max_dim=config.IMAGE_MAX_DIM,
            mode=config.IMAGE_RESIZE_MODE)
            
        patient_id = os.path.splitext(os.path.basename(image_id))[0]

        results = model.detect([image])
        r = results[0]

        out_str = ""
        out_str += patient_id 
        out_str += ","
        assert( len(r['rois']) == len(r['class_ids']) == len(r['scores']) )
        if len(r['rois']) == 0: 
            pass
        else: 
            num_instances = len(r['rois'])
            for i in range(num_instances): 
                if r['scores'][i] > min_conf: 
                    out_str += ' '
                    out_str += str(round(r['scores'][i], 2))
                    out_str += ' '

                    # x1, y1, width, height 
                    x1 = r['rois'][i][1]
                    y1 = r['rois'][i][0]
                    width = r['rois'][i][3] - x1 
                    height = r['rois'][i][2] - y1 
                    bboxes_str = "{} {} {} {}".format(x1*resize_factor, y1*resize_factor, \
                                                      width*resize_factor, height*resize_factor)    
                    out_str += bboxes_str

        file.write(out_str+"\n")

sample_submission_fp = 'Mask_RCNN/submission.csv'
predict(test_image_fps, filepath=sample_submission_fp)

Loading weights from  E:\学习\ml-lessons\lesson3-data\logs\pneumonia20181013T2344\mask_rcnn_pneumonia_0034.h5
Re-starting from epoch 34


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [10:58<00:00,  1.48it/s]
